A recursive solution to Day 19, Part 1 only. If I were less tired, I probably could've taken this to Part 2, but alas, I was pretty tired this day.

In [1]:
from typing import List, Dict


class Rule:
    
    def __init__(self, raw_rule: str):
        self.sub_rule_lists = []
        self.is_expanded = False
        # The following attributes will be added outside of __init__
        self.all_rules = None  # Dict: rule number -> "super" rule
        self.allowed_messages = None
        
        for raw_sub_rule_list in raw_rule.split(" | "):
            sub_rule_list = []
            for sub_rule in raw_sub_rule_list.split():
                if sub_rule in ['"a"', '"b"']:
                    sub_rule_list.append(sub_rule[1:-1])
                    self.is_expanded = True
                else:
                    sub_rule_list.append(int(sub_rule))

            self.sub_rule_lists.append(sub_rule_list)

    def __repr__(self) -> str:
        return f"{Rule}({self.sub_rule_lists}, is_expanded={self.is_expanded})"
    
    def expand(self) -> None:
        if self.is_expanded:
            return
        
        if not self.all_rules:
            raise AttributeError("Expecting `self.all_rules` to be set!")
        

        new_sub_rule_lists = []
        for sub_rule_list in self.sub_rule_lists:
            expanded = [[]]
            
            for entry in sub_rule_list:
                if entry == "a" or entry == "b":
                    # base case
                    additions = [[entry]]
                else:
                    sub_rule = self.all_rules[entry]
                    sub_rule.expand()
                    additions = sub_rule.sub_rule_lists
            
                new_expanded = []
                for exp in expanded:
                    for addition in additions:
                        new_expanded.append(exp + addition)
                expanded = new_expanded
            
            new_sub_rule_lists.extend(expanded)

        self.is_expanded = True
        self.sub_rule_lists = new_sub_rule_lists
        
    def is_match(self, message: str) -> bool:
        self.expand()
        if self.allowed_messages is None:
            self.allowed_messages = set()
            for sub_rule_list in self.sub_rule_lists:
                self.allowed_messages.add("".join(sub_rule_list))
        
        return message in self.allowed_messages


def parse_rules(raw_rules: str) -> Dict[int, Rule]:
    split_raw_rules = raw_rules.splitlines()
    rules = {}
    
    for raw_rule in split_raw_rules:
        number = int(raw_rule.split(": ")[0])
        rule = Rule(raw_rule.split(": ")[1])
        rules[number] = rule

    for rule in rules.values():
        rule.all_rules = rules
    return rules

# Part 1

In [2]:
filename = "day-19-input.txt"

with open(filename) as file:
    raw_rules, raw_messages = file.read().split("\n\n")
    
rules = parse_rules(raw_rules)
messages = raw_messages.splitlines()

rule_zero = rules[0]
rule_zero.expand()

count = 0
for message in messages:
    if rule_zero.is_match(message):
        count += 1

print(f"{count} messages completely match rule 0.")

195 messages completely match rule 0.
